In [3]:
import pandas as pd
from pathlib import Path
from seqeval.scheme import auto_detect, Entities
from seqeval.metrics.sequence_labeling import get_entities

class EntityAnnotator:
    def __init__(self, y_true, y_pred):
        """Initialize the annotator with true and predicted labels."""
        self.y_true = y_true
        self.y_pred = y_pred
        self.scheme = auto_detect(self.y_true, False)
    
    def extract_entities(self):
        """Extract entities based on the scheme detected."""
        self.entities_strict_true = Entities(self.y_true, self.scheme, False)
        self.entities_strict_pred = Entities(self.y_pred, self.scheme, False)
        self.entities_true = get_entities(self.y_true)
        self.entities_pred = get_entities(self.y_pred)

    def process_strict_entities(self, y_true, entities_true, sen_id):
        """Process entities strictly, labeling full spans in a sentence."""
        max_len = len(y_true[sen_id])
        results = ['O'] * max_len
        for idx in range(max_len):
            for entity in entities_true[sen_id]:
                _, t, s, e = entity.to_tuple()
                if s == idx and (e-s) > 0:
                    for i in range(e-s):
                        results[s + i] = t
                elif (e-s) == 0:
                    results[s] = t
        return results
    
    def process_non_strict_entities(self, y_true, sen_id):
        """Process entities non-strictly, marking only the start and end of each entity."""
        max_len = len(y_true[sen_id])
        results = ['O'] * max_len
        for entity in get_entities(y_true[sen_id]):
            t, s, e = entity
            if s == e:
                # If start and end are the same, only mark the start
                results[s] = t
            else:
                # Mark all indices from start to end inclusive
                for i in range(s, e + 1):
                    results[i] = t
        return results

    def process_sentences(self, analysis_data, y_data, entities, label_column, strict=False):
        """Annotate sentences with entity information, either strictly or non-strictly."""
        entity_annotations = []
        for sentence_id, sentence_df in analysis_data.groupby('sentence_ids'):
            if strict: 
                results = self.process_strict_entities(y_data, entities, sentence_id)
            else:
                results = self.process_non_strict_entities(y_data, sentence_id)
            original_series = sentence_df[label_column]
            is_metadata = original_series.apply(lambda x: x not in ['[CLS]', '[SEP]', 'IGNORED'])
            new_series = original_series.copy()
            new_series.loc[is_metadata] = results
            entity_annotations.append(new_series)
        return pd.concat(entity_annotations)

    def annotate_entity_info(self, analysis_data):
        """Add annotated entity information to the analysis data for both true and predicted labels."""
        self.extract_entities()  # Ensure entities are extracted before processing
        analysis_data['strict_true_entities'] = self.process_sentences(analysis_data, self.y_true, self.entities_strict_true.entities, 'true_labels', True)
        analysis_data['strict_pred_entities'] = self.process_sentences(analysis_data, self.y_pred, self.entities_strict_pred.entities, 'pred_labels', True)
        analysis_data['true_entities'] = self.process_sentences(analysis_data, self.y_true, self.entities_true, 'true_labels')
        analysis_data['pred_entities'] = self.process_sentences(analysis_data, self.y_pred, self.entities_pred, 'pred_labels')
        
        return analysis_data


def annotate_error(true_label, pred_label):
        # If both are the same, it's correct (no error)
        if true_label == pred_label:
            return "No Errors"
        
        # Handle cases where one or both labels are 'O'
        if true_label == 'O' and pred_label != 'O':
            return "Chunk"  # False entity predicted
        if true_label != 'O' and pred_label == 'O':
            return "Exclusion"  # Missed entity and chunk boundary
        
        # Extract entity types without position tags (like "B-", "I-")
        true_entity = true_label.split("-")[-1] if "-" in true_label else true_label
        pred_entity = pred_label.split("-")[-1] if "-" in pred_label else pred_label

        # If entity types are different (e.g., LOC vs. PER)
        if true_entity != pred_entity:
            # If both entity type and position (B- vs I-) are wrong
            return "Type and Chunk" if true_label[0] != pred_label[0] else "Type"

        # If entity types are the same but position tags (B- vs I-) are wrong
        return "Chunk"

In [4]:
import pandas as pd
from pathlib import Path

def process_and_update_analysis_data(dataset_name, model_name):
    # Setup file paths
    base_path = Path(f"/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/{dataset_name}_{model_name}/extractions/analysis")
    original_data_path = base_path / "analysis_data.json"
    scrapped_data_path = base_path / "analysis_data-backup.json"
    
    # Read original data
    analysis_data = pd.read_json(original_data_path, lines=True)

    # Process data
    core_data = analysis_data[analysis_data['labels'] != -100].copy()
    y_true = core_data.groupby('sentence_ids')['true_labels'].apply(list).tolist()
    y_pred = core_data.groupby('sentence_ids')['pred_labels'].apply(list).tolist()

    # Modify data
    annotator = EntityAnnotator(y_true, y_pred)  # Assuming EntityAnnotator is defined elsewhere
    updated_analysis_data = annotator.annotate_entity_info(analysis_data)

    # Misalignment checking
    misalignment_mask = updated_analysis_data['pred_entities'] == updated_analysis_data['strict_pred_entities']
    updated_analysis_data['aligned_scheme'] = misalignment_mask

    # Drop unused columns
    updated_analysis_data = updated_analysis_data.drop(['tr_entity', 'pr_entity'], axis=1)

    # Annotate error types
    updated_analysis_data['error_type'] = updated_analysis_data.apply(
        lambda row: annotate_error(row['true_labels'], row['pred_labels']), 
        axis=1
    )

    # Backup original data
    analysis_data.to_json(scrapped_data_path, lines=True, orient='records')
    print("Original data backed up.")

    # Save updated data
    updated_analysis_data.to_json(original_data_path, lines=True, orient='records')
    print("Updated data saved.")


In [5]:
dataset_name = 'ANERCorp_CamelLab'
model_name = 'arabertv02'
process_and_update_analysis_data(dataset_name,  model_name)


Original data backed up.
Updated data saved.


In [6]:
model_name = 'bert'
dataset_name = 'conll2003'
process_and_update_analysis_data(dataset_name,  model_name)

KeyError: "['tr_entity', 'pr_entity'] not found in axis"

# loose

In [ ]:
dataset_name = 'ANERCorp_CamelLab'
model_name = 'arabertv02'
original_data_path = Path(f"/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/{dataset_name}_{model_name}/extractions/analysis/analysis_data.json")
scrapped_data_path = Path(f"/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/{dataset_name}_{model_name}/extractions/analysis/analysis_data-backup.json")
analysis_data = pd.read_json(
	original_data_path,
	lines=True
)


In [3]:
core_data = analysis_data[analysis_data['labels']!= -100].copy()
y_true = core_data.groupby('sentence_ids')['true_labels'].apply(list).tolist()
y_pred = core_data.groupby('sentence_ids')['pred_labels'].apply(list).tolist()

In [4]:
annotator = EntityAnnotator(y_true, y_pred)
updated_analysis_data = annotator.annotate_entity_info(analysis_data)


In [5]:
misalignment_mask = updated_analysis_data['pred_entities'] == updated_analysis_data['strict_pred_entities']
updated_analysis_data['aligned_scheme'] = misalignment_mask

In [6]:
updated_analysis_data['aligned_scheme'].value_counts()

aligned_scheme
True     29665
False       46
Name: count, dtype: int64

In [7]:
updated_analysis_data = updated_analysis_data.drop(['tr_entity', 'pr_entity'], axis=1)

In [8]:
updated_analysis_data['error_type'] = updated_analysis_data.apply(
            lambda row: annotate_error(
                row['true_labels'], 
                row['pred_labels']
                ), 
            axis=1
            )


In [9]:
updated_analysis_data['error_type'].value_counts()

error_type
No Errors         29005
Exclusion           311
Chunk               236
Type                124
Type and Chunk       35
Name: count, dtype: int64

In [12]:
# I have saved the modified data in the same name as the original and scrapped the other ones.
updated_analysis_data.to_json(
	original_data_path,
	lines=True, orient='records'
)


analysis_data.to_json(
	scrapped_data_path,
	lines=True, orient='records'
)


# Examples

In [12]:
updated_analysis_data[['true_entities', 'strict_true_entities', 'tr_entity', 'true_labels']].iloc[200:250]

,true_entities,strict_true_entities,tr_entity,true_labels
200,O,O,O,O
201,LOC,LOC,LOC,B-LOC
202,O,O,O,O
203,O,O,O,O
204,O,O,O,O
205,O,O,O,O
206,O,O,O,O
207,O,O,O,O
208,O,O,O,O
209,O,O,O,O


In [20]:
updated_analysis_data[updated_analysis_data['true_entities'] != updated_analysis_data['strict_true_entities']][['true_entities', 'strict_true_entities', 'tr_entity', 'true_labels', 'agreements']].head(50)

,true_entities,strict_true_entities,tr_entity,true_labels,agreements
379,PERS,O,PERS,I-PERS,False
415,ORG,O,ORG,I-ORG,False
1171,PERS,O,PERS,I-PERS,False
1196,MISC,O,MISC,I-MISC,False
1197,MISC,O,MISC,I-MISC,False
1198,MISC,O,MISC,I-MISC,False
1207,PERS,O,PERS,I-PERS,False
2810,ORG,O,ORG,I-ORG,False
2928,LOC,O,LOC,I-LOC,False
3208,PERS,O,PERS,I-PERS,False


In [22]:
updated_analysis_data[updated_analysis_data['sentence_ids'] == 130]

,sentence_ids,token_positions,words,tokens,word_pieces,core_tokens,true_labels,token_selector_id,pred_labels,agreements,...,prediction_entropy,prediction_max_entropy,confidence,variability,pre_x,pre_y,strict_true_entities,strict_pred_entities,true_entities,pred_entities
4560,130,0,[CLS],[CLS],[CLS],[CLS],[CLS],[CLS]@#0@#130,[CLS],True,...,0.003089,3.169925,0.999807,0.314201,2.536585,-0.162095,[CLS],[CLS],[CLS],[CLS]
4561,130,1,فيينا,فيينا,[فيينا],فيينا,B-LOC,فيينا@#1@#130,O,False,...,0.004867,3.169925,0.999685,0.314158,1.048240,4.441457,LOC,O,LOC,O
4562,130,2,المعلومات,المعلومات,[المعلومات],المعلومات,O,المعلومات@#2@#130,O,True,...,0.001409,3.169925,0.999920,0.314241,0.268964,12.650270,O,O,O,O
4563,130,3,التي,التي,[التي],التي,O,التي@#3@#130,O,True,...,0.000854,3.169925,0.999954,0.314254,-3.795791,14.442162,O,O,O,O
4564,130,4,بثها,بثها,[بثها],بثها,O,بثها@#4@#130,O,True,...,0.001411,3.169925,0.999921,0.314242,-2.035390,12.893243,O,O,O,O
4565,130,5,موقع,موقع,[موقع],موقع,O,موقع@#5@#130,O,True,...,0.003905,3.169925,0.999746,0.314180,1.437478,12.406041,O,O,O,O
4566,130,6,غلوبو,غلوب,"[غلوب, ##و]",غلوب,B-ORG,غلوب@#6@#130,B-ORG,True,...,0.524218,3.169925,0.924509,0.287821,0.794667,7.322890,ORG,ORG,ORG,ORG
4567,130,7,غلوبو,##و,"[غلوب, ##و]",IGNORED,IGNORED,IGNORED@#7@#130,IGNORED,True,...,1.640223,3.169925,0.654458,0.198654,0.597000,8.079787,IGNORED,IGNORED,IGNORED,IGNORED
4568,130,8,على,على,[على],على,O,على@#8@#130,O,True,...,0.002080,3.169925,0.999880,0.314227,-2.472588,15.070498,O,O,O,O
4569,130,9,شبكة,شبكة,[شبكة],شبكة,O,شبكة@#9@#130,O,True,...,0.004785,3.169925,0.999698,0.314163,-1.828349,13.865242,O,O,O,O


In [352]:
updated_analysis_data[updated_analysis_data['pred_entities'] != updated_analysis_data['strict_pred_entities']][['pred_entities', 'strict_pred_entities', 'pred_labels', 'true_labels']]

,pred_entities,strict_pred_entities,pred_labels,true_labels
1181,ORG,O,I-ORG,I-ORG
1182,ORG,O,I-ORG,I-ORG
1183,ORG,O,I-ORG,I-ORG
1210,ORG,O,I-ORG,I-ORG
1384,LOC,O,I-LOC,I-ORG
3266,PERS,O,I-PERS,I-PERS
4324,ORG,O,I-ORG,I-ORG
4437,MISC,O,I-MISC,O
4580,MISC,O,I-MISC,I-MISC
4739,ORG,O,I-ORG,I-ORG


In [355]:
analysis_data[analysis_data['sentence_ids'] == 33][['pred_entities', 'strict_pred_entities', 'pred_labels', 'true_labels']]

,pred_entities,strict_pred_entities,pred_labels,true_labels
1176,[CLS],[CLS],[CLS],[CLS]
1177,O,O,O,O
1178,O,O,O,O
1179,O,O,O,O
1180,O,O,O,B-ORG
1181,ORG,O,I-ORG,I-ORG
1182,ORG,O,I-ORG,I-ORG
1183,ORG,O,I-ORG,I-ORG
1184,O,O,O,O
1185,O,O,O,O


In [ ]:
for e, e_df in updated_analysis_data.groupby('error_type'):

    display(e_df.sample(20))